In [56]:
import pandas as pd
df = pd.read_csv('ohou_urls.csv')
df = df[1400:2100]
df = df.reset_index(drop=True)

df['제목'] = 0 # 집들이 글 제목
df['작성일자'] = 0
df['평수'] = 0 # 평수 (1-9/10/20/30/40/50/60/70이상)
df['가족형태'] = 0 # 가족형태 (싱글라이프/신혼부부/아기가있는집/취학자녀가있는집/부모님과함꼐사는집/기타)
df['공간'] = 0 # 공간 (원룸&오피스텔/아파트/빌라&연합/단독주택/사무공간/상업공간/기타)
df['스타일'] = 0 # 스타일 (모던/미니멀&심플/내추럴/북유럽/빈티지&레트로/클래식&앤틱/프렌치&프로방스/러블리&로맨틱/인더스트리얼/한국&아시아/유니크&믹스매치)
df['세부공사'] = 0 # 세부공사 (헤링본 마루/대리석 바닥/ 원목마루/포세린타일/폴링싱타일/주방리모델링/조명시공/폴딩도어/중문/가벽&파티션/슬라이딩도어/아트월/발코니확장)
df['작업'] = 0 # 작업 (셀프,DIY/반셀프/전문가)
df['지역'] = 0 # 지역
df['분야'] = 0 # 분야 (리모델링/홈스타일링/부분공사/건축)
df['예산'] = 0 # 예산 (1백미만/1백/2백/3백/4백/5백/1천/2천/3천/4천이상)
df['스크랩'] = 0
df['좋아요'] = 0
df['댓글'] = 0
df['조회'] = 0


In [57]:
df.head() # 인덱스 번호 변경 확인

,url,제목,작성일자,평수,가족형태,공간,스타일,세부공사,작업,지역,분야,예산,스크랩,좋아요,댓글,조회
0,https://ohou.se/projects/77516/detail?affect_t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,https://ohou.se/projects/76807/detail?affect_t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,https://ohou.se/projects/76630/detail?affect_t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,https://ohou.se/projects/76256/detail?affect_t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,https://ohou.se/projects/73722/detail?affect_t...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [58]:
df.columns[2:]

Index(['작성일자', '평수', '가족형태', '공간', '스타일', '세부공사', '작업', '지역', '분야', '예산',
       '스크랩', '좋아요', '댓글', '조회'],
      dtype='object')

In [59]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as BS
import pandas as pd
from IPython.display import display

chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = chrome_options)
# 페이지가 모두 로드될 때까지 3초 기다려주도록 처리
driver.implicitly_wait(3)

for idx, url in enumerate(df['url']):
    # Selenium을 통해 실제 크롬창에서 접속한 것과 동일하게 해당 URL내용을 가져옴
    driver.get(df['url'][idx])
    # 소스를 읽어 BeautifulSoup를 이용해 파싱
    html = driver.page_source
    soup = BS(html, 'html.parser')
    
    title = soup.select('body > div.page > div > div > main > div.content-detail-content-section > div.content-detail-content-section__content > header > h1')
    date = soup.select('body > div.page > div > div > main > div.content-detail-content-section > div.content-detail-content-section__content > header > div.content-detail-header__bottom > a > div.content-detail-header__author-date')
    texts = soup.select('body > div.page > div > div > main > div.content-detail-content-section > div.content-detail-content-section__content > section > div > div.project-detail-foldable__content > dl > div')
    reactions = soup.select('body > div.page > div > div > main > div.content-detail-footer > dl')
    
    df['제목'][idx] = title[0].text
    df['작성일자'][idx] = date[0].text
    
    text_dic = {}
    for text in texts:
        text_dic[text.find('dt').text] = text.find('dd').text
    for react in reactions[0].find_all('div'):
        text_dic[react.find('dt').text] = react.find('dd').text

    for col in df.columns[2:]: 
        if col in text_dic:
            df[col][idx] = text_dic[col]
    print(df['url'][idx])
driver.close()

C:\Users\skygg\AppData\Local\Temp\ipykernel_33760\2774447752.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['제목'][idx] = title[0].text
C:\Users\skygg\AppData\Local\Temp\ipykernel_33760\2774447752.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['작성일자'][idx] = date[0].text
C:\Users\skygg\AppData\Local\Temp\ipykernel_33760\2774447752.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col][idx] = text_dic[col]


https://ohou.se/projects/77516/detail?affect_type=ProjectSelfIndex&affect_id=1400
https://ohou.se/projects/76807/detail?affect_type=ProjectSelfIndex&affect_id=1401
https://ohou.se/projects/76630/detail?affect_type=ProjectSelfIndex&affect_id=1402
https://ohou.se/projects/76256/detail?affect_type=ProjectSelfIndex&affect_id=1403
https://ohou.se/projects/73722/detail?affect_type=ProjectSelfIndex&affect_id=1404
https://ohou.se/projects/78200/detail?affect_type=ProjectSelfIndex&affect_id=1405


KeyboardInterrupt: 

In [61]:
nae_house = df
nae_house.to_csv('1400-2100.csv')

In [62]:
nae_house.head()

,url,제목,작성일자,평수,가족형태,공간,스타일,세부공사,작업,지역,분야,예산,스크랩,좋아요,댓글,조회
0,https://ohou.se/projects/77516/detail?affect_t...,인디핑크 vs 화이트! 색상에 따른 오피스텔의 변화,02. 13,10평,싱글라이프,원룸&오피스텔,내추럴,0,셀프•DIY,서울특별시,홈스타일링,0,"1,599",340,32,"49,697"
1,https://ohou.se/projects/76807/detail?affect_t...,"단정해 보이는 아파트의 비밀은, 화이트 & 수납의 힘!",02. 13,33평,취학 자녀가 있는 집,아파트,내추럴,0,전문가,서울특별시 서대문구,리모델링,0,"1,830",474,44,"63,507"
2,https://ohou.se/projects/76630/detail?affect_t...,턴키 올리모델링 꿀팁이 가득한 몽글몽글 신혼집,02. 13,32평,신혼부부,아파트,모던,"주방리모델링, 조명시공, 중문, 가벽&파티션, 아트월, 발코니확장",전문가,0,리모델링,"7,300만원","1,013",228,50,"40,842"
3,https://ohou.se/projects/76256/detail?affect_t...,"유럽 집의 기억을 담아, 아파트 반셀프 인테리어 도전기",02. 13,34평,취학 자녀가 있는 집,아파트,북유럽,0,반셀프,충청남도 천안시,홈스타일링,"2,700만원",861,206,42,"36,008"
4,https://ohou.se/projects/73722/detail?affect_t...,INFP 집순이의 구축 아파트 빈티지 셀프 인테리어,02. 13,32평,싱글라이프,아파트,내추럴,0,셀프•DIY,0,홈스타일링,100만원,736,157,17,"21,800"
